In [1]:
pip install shared_utils

ERROR: Could not find a version that satisfies the requirement shared_utils (from versions: none)
ERROR: No matching distribution found for shared_utils
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importing required packages 
import pandas as pd
import geopandas as gpd
import gcsfs
from calitp_data_analysis import get_fs
fs = get_fs()
import numpy as np
from calitp_data_analysis.sql import get_engine
db_engine = get_engine()

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [3]:
# read in and concatenate all geoparquets
fs_list = fs.ls(f"{GCS_FILE_PATH}tool_data_2025/")

In [4]:
filelist = []
for f in fs_list[1:]:
    try: 
        test_pqt = pd.read_parquet(f"gs://{f}")
        filelist.append(f)
    except:
        print(f"error on {f.split('tool_data_2025/')}")

In [5]:
df = (pd.concat(
        pd.read_parquet(f"gs://{f}")
        for f in filelist
        )
    )

In [6]:
wkd_coeff = np.array([-0.16286, 0.00010, -0.00002, 0.02713, -0.01183, -0.03907, -0.02564, 0.01571])
sat_coeff = np.array([-0.13482,	0.00012, -0.00002,	0.0293,	-0.01627, -0.03214,	-0.02826, 0.02126])
sun_coeff = np.array([-0.1087,	0.00012, -0.00003,	0.02644, -0.01624,	-0.03621, -0.04081,	0.02355])

In [7]:
# NTD Ridership data 
with db_engine.connect() as connection:
    query = """
        SELECT
            key, ntd_id, agency, period_year_month, mode, uza_name, upt
        FROM 
            cal-itp-data-infra.mart_ntd_ridership.fct_complete_monthly_ridership_with_adjustments_and_estimates
    """
    ridership_data= pd.read_sql(query, connection)

In [8]:
ridership_data.dtypes

key                   object
ntd_id                object
agency                object
period_year_month     object
mode                  object
uza_name              object
upt                  float64
dtype: object

In [9]:
# Convert to Period (monthly)
ridership_data['period_year_month'] = pd.to_datetime(ridership_data['period_year_month'], format='%Y-%m').dt.to_period('M')


In [10]:
ridership_data.head(5)

,key,ntd_id,agency,period_year_month,mode,uza_name,upt
0,768154404f12b3a096247a985ebfc1a8,20120,City of Glens Falls,2005-03,OR,"Glens Falls, NY",0.00
1,1e1188b7d77e5ebd8966ba22efd5e4e6,20120,City of Glens Falls,2005-02,OR,"Glens Falls, NY",0.00
2,e5fcb9a9c434254d07dc77306f692451,20120,City of Glens Falls,2005-08,OR,"Glens Falls, NY",13259.00
3,a389d3dfa66a82426209e76af8a00729,20120,City of Glens Falls,2005-12,OR,"Glens Falls, NY",0.00
4,fb66f457e8a0cbf48bb4e09a1b2ea4aa,20120,City of Glens Falls,2005-01,OR,"Glens Falls, NY",0.00


In [11]:
# Step 1: Filter by year-month range (as strings)
filtered_ridership_data = ridership_data[
    (ridership_data['period_year_month'] >= '2021-10') &
    (ridership_data['period_year_month'] <= '2022-09')
]

# Step 2: Filter modes ending in "B"
filtered_ridership_data = filtered_ridership_data[filtered_ridership_data['mode'].str.endswith('B')]

# Step 3: Group and sum UPT
grouped_ridership_data = (
    filtered_ridership_data
    .groupby('ntd_id', as_index=False)
    .agg(ntd_ridership=('upt', 'sum'))
)

In [12]:
grouped_ridership_data.head(5)

,ntd_id,ntd_ridership
0,00001,61493156.00
1,00002,6158722.00
2,00003,4700062.00
3,00005,940548.00
4,00006,478335.00


Apply coefficients to data based on day type

In [13]:
#Preprocess stop level data 
def apply_coefficients(df, daytype, coeff_list):
    df_day = df[df['daytype'] == daytype].copy()
    cols_to_scale = ['n_routes', 'pop_density', 'job_density', 'pct_not_us_citizen_pop',
                     'pct_youth_pop', 'pct_seniors_pop', 'pct_pop_workers_no_car', 'pct_poverty']
    df_day[cols_to_scale] = df_day[cols_to_scale].multiply(coeff_list)
    return df_day

In [14]:
# Estimate Baseline Ridership
def compute_model_ridership(df, coefficient, constant, variance):
    df['control_vars_sum'] = df[['n_routes', 'pop_density', 'job_density',
                                 'pct_not_us_citizen_pop', 'pct_youth_pop', 
                                 'pct_seniors_pop', 'pct_pop_workers_no_car', 
                                 'pct_poverty']].sum(axis=1)
    
    df['control_vars_factor'] = df['control_vars_sum'] + (df['n_trips'] * coefficient) + constant
    df['model_est_ridership'] = np.exp(df['control_vars_factor'] + (variance / 2))
    return df

In [15]:
# Remove outliers
def remove_outliers(df, quantile=0.99):
    cutoff = df['model_est_ridership'].quantile(quantile)
    return df[df['model_est_ridership'] < cutoff].copy()

In [16]:
def aggregate_system_model(df):
    return df.groupby(['schedule_gtfs_dataset_key', 'portfolio_organization_name', 'ntd_id_2022'], 
                      as_index=False)['model_est_ridership'].sum().rename(
        columns={'model_est_ridership': 'sys_model_est_ridership'})

In [17]:
def join_external_data(sys_df, ntd_ridership_df, daytype_count):
    sys_df = sys_df.merge(
        ntd_ridership_df, 
        left_on='ntd_id_2022', 
        right_on='ntd_id', 
        how='left'
    )
    sys_df['ntd_ridership_daytype'] = sys_df['ntd_ridership'] * (daytype_count / 365)
    return sys_df

In [18]:
def compute_median_overprediction(sys_df):
    sys_df['model_over_ntd'] = sys_df['sys_model_est_ridership'] / sys_df['ntd_ridership_daytype']
    return sys_df['model_over_ntd'].median()

In [19]:
def scale_ridership(df, sys_df, median_overpred):
    df = df.merge(
        sys_df[['schedule_gtfs_dataset_key', 'portfolio_organization_name', 'ntd_id_2022', 'sys_model_est_ridership', 'ntd_ridership_daytype']],
        on=['schedule_gtfs_dataset_key', 'portfolio_organization_name', 'ntd_id_2022'], how='left'
    )
    
    def scaled_rider(row):
        if pd.notnull(row['ntd_ridership_daytype']) and row['ntd_ridership_daytype'] > 0:
            return row['model_est_ridership'] / (row['sys_model_est_ridership'] / row['ntd_ridership_daytype'])
        else:
            return row['model_est_ridership'] / median_overpred

    df['ntd_scaled_ridership'] = df.apply(scaled_rider, axis=1)
    return df

In [20]:
def simulate_added_trips(df, range_trips):
    df['route_list_dups'] = df['route_list_string'].str.split(',')
    df['route_list_cleaned'] = df['route_list_dups'].apply(
        lambda routes: [r.split('_')[0] for r in routes]
    )
    df['route_name'] = df['route_list_cleaned'].apply(np.unique)
    df = df.explode('route_name')
    df['n_addtl_trips'] = [range_trips] * len(df)
    df = df.explode('n_addtl_trips')
    return df

In [21]:
def estimate_new_riders(df, coefficient):
    coeff_pct = np.exp(coefficient) - 1
    df['n_addtl_riders'] = df['ntd_scaled_ridership'] * coeff_pct * df['n_addtl_trips']
    return df[['schedule_gtfs_dataset_key', 'portfolio_organization_name', 'ntd_id_2022', 'daytype', 'route_name', 'stop_id', 'stop_name',
               'n_addtl_trips', 'n_addtl_riders']]

In [22]:
def ridership_model(df, daytype, coeff_list, coefficient, constant, variance, 
                    daytype_count, range_trips, NTD_ridership_clean):

    #Prepare and model ridership
    df1 = apply_coefficients(df, daytype, coeff_list)
    df2 = compute_model_ridership(df1, coefficient, constant, variance)
    df3 = remove_outliers(df2)
    
    #System-level scaling based on real data
    sys_model = aggregate_system_model(df3)
    sys_model = join_external_data(sys_model, NTD_ridership_clean, daytype_count)
    median_overpred = compute_median_overprediction(sys_model)
    
    # STEP 7–9: Stop-level scaling, simulating added trips, estimating new riders
    df4 = scale_ridership(df3, sys_model, median_overpred)
    df5 = simulate_added_trips(df4, range_trips)
    df_final = estimate_new_riders(df5, coefficient)
    
    return df_final

In [23]:
df_weekday = ridership_model(
    df,
    daytype="Weekday",
    coeff_list=wkd_coeff,
    coefficient=0.0200865,
    constant=6.8721538,
    variance=2.434,
    daytype_count=261,
    range_trips=list(range(0, 21)),
    NTD_ridership_clean=grouped_ridership_data
)

df_saturday = ridership_model(
    df,
    daytype="Saturday",
    coeff_list=sat_coeff,
    coefficient=0.0262958,
    constant=4.2261843,
    variance=3.007,
    daytype_count=52,
    range_trips=list(range(0, 21)),
    NTD_ridership_clean=grouped_ridership_data
)

df_sunday = ridership_model(
    df,
    daytype="Sunday",
    coeff_list=sun_coeff,
    coefficient=0.0263988,
    constant=3.9734396,
    variance=3.119,
    daytype_count=52,
    range_trips=list(range(0, 21)),
    NTD_ridership_clean=grouped_ridership_data
)

In [24]:
df_weekday.head(5)

,schedule_gtfs_dataset_key,portfolio_organization_name,ntd_id_2022,daytype,route_name,stop_id,stop_name,n_addtl_trips,n_addtl_riders
0,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,0,0.00
0,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,1,853.79
0,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,2,1707.58
0,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,3,2561.38
0,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,4,3415.17


In [25]:
df_all = pd.concat([df_weekday,df_saturday,df_sunday], ignore_index=True)

In [26]:
df_all.shape

(7465605, 9)

In [27]:
df_all.head(5)

,schedule_gtfs_dataset_key,portfolio_organization_name,ntd_id_2022,daytype,route_name,stop_id,stop_name,n_addtl_trips,n_addtl_riders
0,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,0,0.00
1,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,1,853.79
2,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,2,1707.58
3,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,3,2561.38
4,ad942f5430e5cad0da5c71953b058742,Golden Empire Transit District,90004,Weekday,21,10,Mount Vernon & University,4,3415.17


In [28]:
df_all.to_csv(f"{GCS_FILE_PATH}AHSC_analysis_table_2025.csv", index=False)

In [29]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7465605 entries, 0 to 7465604
Data columns (total 9 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   schedule_gtfs_dataset_key    object
 1   portfolio_organization_name  object
 2   ntd_id_2022                  object
 3   daytype                      object
 4   route_name                   object
 5   stop_id                      object
 6   stop_name                    object
 7   n_addtl_trips                object
 8   n_addtl_riders               object
dtypes: object(9)
memory usage: 512.6+ MB
